In [23]:
import pandas as pd
import numpy as np
from mapbox import MapMatcher

In [24]:
df = pd.read_csv("C:/Users/hysir/Desktop/研究室/kyosaku_work/result/result20220123_Wakkanai.csv")
latlon = []
for lat, lon in zip(df["latitude"], df["longitude"]):
    latlon.append([lon,lat])
print(latlon)

imgnamex = list(df["image_name"])
print(imgnamex)
imglist = []
for imgname in imgnamex:
    img = "20"+imgname[6:8]+"-"+imgname[8:10]+"-"+imgname[10:12]+"T"+imgname[13:15]+":"+imgname[15:17]+":"+imgname[17:19]+"Z"
    imglist.append(img)

print(imglist)

[[141.34261166666664, 42.966771666666666], [141.34261166666664, 42.966771666666666], [141.34261166666664, 42.966771666666666], [141.34261166666664, 42.966771666666666], [141.34261166666664, 42.966771666666666], [141.34261, 42.966771666666666], [141.34261, 42.966773333333336], [141.3425983333333, 42.966785], [141.3425983333333, 42.966785], [141.34259666666668, 42.966785], [141.34259666666668, 42.96678833333333], [141.34259666666668, 42.96678833333333], [141.34259666666668, 42.96678833333333], [141.34259666666668, 42.96678833333333], [141.34259500000002, 42.96678833333333], [141.34259500000002, 42.96679000000001], [141.34257833333334, 42.966796666666674], [141.34257833333334, 42.966796666666674], [141.34257833333334, 42.966796666666674], [141.34257833333334, 42.966796666666674], [141.34257833333334, 42.96679833333333], [141.342585, 42.9668], [141.34260666666663, 42.96679833333333], [141.34261, 42.966796666666674], [141.34261, 42.966796666666674], [141.34261, 42.966796666666674], [141.342

In [25]:
service = MapMatcher(access_token="pk.eyJ1IjoicnlvMTFoeXMiLCJhIjoiY2w3OGZ5a3F4MDk4YjN2czlzbGFwMGd1aCJ9.ebDE0Nqg_VaPzTF57ou66A")

In [26]:
#map matching api は一度に100件までしか処理できないため分割処理
corrected=[]
dist=0
for i in range(len(latlon)//100+1):
    #apiの入力形式に合わせる
    line={}
    line["type"]="Feature"
    line["properties"]={"coordTimes":imglist[i*100:(i+1)*100]}
    line["geometry"]={"type": "LineString","coordinates":latlon[i*100:(i+1)*100]}
    
    response = service.match(line, profile='mapbox.driving')
    print(i,response)
    #結果を統合
    try:
        for i in response.geojson()['features']:
            corrected.extend(i["properties"]["matchedPoints"])
            dist+=i["properties"]["distance"]
    except KeyError:
        pass

0 <Response [200]>
1 <Response [200]>
2 <Response [200]>
3 <Response [200]>
4 <Response [200]>
5 <Response [200]>
6 <Response [200]>
7 <Response [200]>
8 <Response [200]>
9 <Response [200]>
10 <Response [200]>
11 <Response [200]>
12 <Response [200]>
13 <Response [200]>
14 <Response [200]>
15 <Response [200]>
16 <Response [200]>
17 <Response [200]>
18 <Response [200]>
19 <Response [200]>
20 <Response [200]>
21 <Response [200]>
22 <Response [200]>
23 <Response [200]>
24 <Response [200]>
25 <Response [200]>
26 <Response [200]>
27 <Response [200]>
28 <Response [200]>
29 <Response [200]>
30 <Response [200]>
31 <Response [200]>
32 <Response [200]>
33 <Response [200]>
34 <Response [200]>
35 <Response [200]>
36 <Response [200]>
37 <Response [200]>
38 <Response [200]>
39 <Response [200]>


In [27]:
#foliumに入力するために緯度経度を逆に
np_corrected=np.array(corrected)
re_corrected=np.zeros_like(np_corrected)
re_corrected[:,1]=np_corrected[:,0]
re_corrected[:,0]=np_corrected[:,1]

In [28]:
print(corrected)

[[141.342636, 42.96677], [141.342636, 42.96677], [141.342636, 42.96677], [141.342636, 42.96677], [141.342636, 42.96677], [141.342636, 42.96677], [141.342636, 42.966771], [141.342638, 42.966782], [141.342638, 42.966782], [141.342638, 42.966782], [141.342638, 42.966785], [141.342638, 42.966785], [141.342638, 42.966785], [141.342638, 42.966785], [141.342638, 42.966785], [141.342638, 42.966787], [141.342639, 42.966792], [141.342639, 42.966792], [141.342639, 42.966792], [141.342639, 42.966792], [141.342639, 42.966793], [141.342639, 42.966796], [141.342639, 42.966796], [141.342639, 42.966795], [141.342639, 42.966795], [141.342639, 42.966795], [141.342639, 42.966795], [141.342639, 42.966795], [141.342639, 42.966796], [141.342639, 42.966796], [141.342639, 42.966795], [141.34264, 42.966797], [141.342641, 42.966803], [141.342641, 42.966807], [141.342641, 42.966807], [141.342641, 42.966807], [141.342641, 42.966807], [141.342641, 42.966808], [141.342641, 42.966808], [141.342641, 42.966808], [141.3

In [29]:
correct = pd.DataFrame(corrected)

correct.to_csv("matching20220123.csv")
prematch = pd.DataFrame(latlon)
prematch.to_csv("prematch20220123.csv")